# Web scrapping cities data 

## creating a generic read from the url and target the area to be parsed

In [1]:
from bs4 import BeautifulSoup as bsoup
from urllib.request import urlopen
import re
urlbase = 'https://en.wikipedia.org'
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'


def readUrl(url):
    html = urlopen(url)
    bsObj = bsoup(html.read(),'html.parser')
    return bsObj

def infoBoxScraper(link):
    bsObj = readUrl(link)
    target_table = bsObj.find_all('table',{"class":"infobox"})[0]
    return target_table


Here we are calling all the main url and using beautifull soup library to detect the target table 

In [2]:
main_page = readUrl(url)
# locating the target page into html formate
table = main_page.find_all('table',{"class":"wikitable sortable"})[0]
# locating target table which contains all cities name

rows = table.find_all('tr')


Now the we understand the data and each cells how data is stored and formulate our techniques to extract the necessary information. There is always a pattern in an the same columns

In [3]:
data={'city':[], 'state':[], 'population_2018':[], 'population_2010':[], 'change':[],
      'area_in_miles':[], 'area_in_kms':[], 'population_density_in_miles':[], 'population_density_in_kms':[],
     'location':[],'wiki_url':[],'mayor':[],'website':[]}
header = (list) (data.keys())
for row in rows[1:]:
    cells = row.find_all('td')
    
    if cells[1].i and cells[1].i.a:
        wikiurl = urlbase+cells[1].i.a['href']
    else :
        wikiurl = urlbase+cells[1].a['href']
    data['wiki_url'].append(wikiurl)
    
    for i in range(1,len(cells)):
        cell = cells[i]
        if i == 1:
            while str(cell)[0] == '<':
                cell = cell.contents[0]
            data[header[i-1]].append(cell)
            print(cell,end=", ")
        elif i == 7 or i==9:
            nums = re.findall('[\.\d]+',re.sub(',','',cell.contents[0]))
            if len(nums)>0:
                cell = float(nums[0])
            else:
                cell = None
            data[header[i-1]].append(cell)
            print(cell,end=", ")
        elif i == 10:
            res = ''
            res +=cell.contents[0].contents[0].contents[0].contents[0].contents[0].contents[0].contents[0]+' '
            res += cell.contents[0].contents[0].contents[0].contents[0].contents[0].contents[2].contents[0]
            data[header[i-1]].append(res)
            print(res)
        else:
            while str(cell)[0] == '<':

                if cell.contents[-1] == '\n' :
                    cell = cell.contents[-2]
                else:
                    cell = cell.contents[-1]
            cell = cell.strip('\n')
            
            if i in [3,4,5,6,8]:
                nums = re.findall('[\.\d]+',re.sub(',','',cell))
                if len(nums)>0:
                    cell = float(nums[0])
                else:
                    cell = None
            data[header[i-1]].append(cell)
            print(cell,end=", ")
    
    #  here we want to use the information learned from the wiki links of each cities and 
    #  use it to navigate to infobox of each city
    table = infoBoxScraper(wikiurl)
    website = None
    mayor = None
    for cell in table.contents[0].contents:
        var = (re.sub('<[^<]+?>',"****",str(cell)))
        #  here we assume **** would be a unique pattern and not consist in the infobox data
        info = [x for x in (var.split('****')) if x !=""]
        if 'Website' in info:
            info.remove('Website')
            for i in info:
                m = re.findall('[\w]+\.[\w\./-]+',i)
                if len(m)>0:
                    website = m[0]
                    break
            # many time the info box stores a href link which we are unable to detect             
            print('website found',website)
        elif 'Mayor' in info:
            info.pop(0)
            info.remove('Mayor')
            mayor = info[0]
            print('Mayor found ',mayor )
    data['mayor'].append(mayor)
    data['website'].append(website)
            
        

New York City, New York, 8398748.0, 8175133.0, 2.74, 301.5, 780.9, 28317.0, 10933.0, 40°39′49″N 73°56′19″W
Mayor found  Bill de Blasio
website found NYC.gov
Los Angeles, California, 3990456.0, 3792621.0, 5.22, 468.7, 1213.9, 8484.0, 3276.0, 34°01′10″N 118°24′39″W
Mayor found  Eric Garcetti
website found None
Chicago, Illinois, 2705994.0, 2695598.0, 0.39, 227.3, 588.7, 11900.0, 4600.0, 41°50′15″N 87°40′54″W
Mayor found  Lori Lightfoot
website found None
Houston, Texas, 2325502.0, 2100263.0, 10.72, 637.5, 1651.1, 3613.0, 1395.0, 29°47′12″N 95°23′27″W
Mayor found  Sylvester Turner
website found houstontx.gov
Phoenix, Arizona, 1660272.0, 1445632.0, 14.85, 517.6, 1340.6, 3120.0, 1200.0, 33°34′20″N 112°05′24″W
Mayor found  Kate Gallego
website found None
Philadelphia, Pennsylvania, 1584138.0, 1526006.0, 3.81, 134.2, 347.6, 11683.0, 4511.0, 40°00′34″N 75°08′00″W
Mayor found  Jim Kenney
website found www.phila.gov
San Antonio, Texas, 1532233.0, 1327407.0, 15.43, 461.0, 1194.0, 3238.0, 1250.0, 

Mayor found  Miguel A. Pulido
website found ana.org
Riverside, California, 330063.0, 303871.0, 8.62, 81.2, 210.3, 3999.0, 1544.0, 33°56′17″N 117°23′36″W
Mayor found  City council
website found None
Corpus Christi, Texas, 326554.0, 305215.0, 6.99, 174.6, 452.2, 1866.0, 720.0, 27°45′15″N 97°10′24″W
Mayor found  City council
website found None
Lexington, Kentucky, 323780.0, 295803.0, 9.46, 283.6, 734.5, 1123.0, 434.0, 38°02′27″N 84°27′30″W
website found None
Stockton, California, 311178.0, 291707.0, 6.67, 61.7, 159.8, 4977.0, 1922.0, 37°58′35″N 121°18′48″W
Mayor found  Michael Tubbs
website found None
Henderson, Nevada, 310390.0, 257729.0, 20.43, 104.7, 271.2, 2798.0, 1080.0, 36°00′35″N 115°02′09″W
Mayor found  Debra March
website found None
Saint Paul, Minnesota, 307695.0, 285068.0, 7.94, 52.0, 134.7, 5815.0, 2245.0, 44°56′56″N 93°06′15″W
Mayor found  Melvin Carter
website found www.stpaul.gov
St. Louis, Missouri, 302838.0, 319294.0, 5.15, 62.0, 160.6, 5023.0, 1939.0, 38°38′09″N 90°14′41

Mayor found  Rosalynn Bliss
website found None
Amarillo, Texas, 199924.0, 190695.0, 4.84, 101.4, 262.6, 1968.0, 760.0, 35°12′00″N 101°49′49″W
Mayor found  Ginger Nelson
website found None
Yonkers, New York, 199663.0, 195976.0, 1.88, 18.0, 46.6, 11156.0, 4307.0, 40°56′45″N 73°52′03″W
Mayor found  Mike Spano
website found www.yonkersny.gov
Aurora, Illinois, 199602.0, 197899.0, 0.86, 44.9, 116.3, 4479.0, 1729.0, 41°45′49″N 88°17′24″W
website found None
Montgomery, Alabama, 198218.0, 205764.0, 3.67, 159.8, 413.9, 1252.0, 483.0, 32°20′50″N 86°15′58″W
Mayor found  Todd Strange
website found None
Akron, Ohio, 198006.0, 199110.0, 0.55, 62.0, 160.6, 3188.0, 1231.0, 41°04′50″N 81°31′17″W
website found None
Little Rock, Arkansas, 197881.0, 193524.0, 2.25, 118.7, 307.4, 1673.0, 646.0, 34°43′31″N 92°21′31″W
Mayor found  Frank Scott Jr.
website found None
Huntsville, Alabama, 197318.0, 180105.0, 9.56, 213.4, 552.7, 905.0, 349.0, 34°41′56″N 86°40′23″W
Mayor found  Tommy Battle
website found None
Augu

Mayor found  City Council
website found None
Naperville, Illinois, 148304.0, 141853.0, 4.55, 38.7, 100.2, 3802.0, 1468.0, 41°44′57″N 88°09′43″W
website found None
Joliet, Illinois, 148099.0, 147433.0, 0.45, 64.4, 166.8, 2302.0, 889.0, 41°31′04″N 88°08′56″W
website found www.cityofjoliet.info
Bellevue, Washington, 147599.0, 122363.0, 20.62, 33.5, 86.8, 4221.0, 1630.0, 47°35′52″N 122°09′23″W
Mayor found  John Chelminiak
website found www.bellevuewa.gov
Rockford, Illinois, 146526.0, 152871.0, 4.15, 63.5, 164.5, 2325.0, 898.0, 42°15′32″N 89°03′53″W
Mayor found  Thomas McNamara (D)
website found None
Savannah, Georgia, 145862.0, 136286.0, 7.03, 103.6, 268.3, 1417.0, 547.0, 32°00′09″N 81°09′13″W
Mayor found  Eddie DeLoach (
website found savannahga.gov
Paterson, New Jersey, 145627.0, 146199.0, 0.39, 8.4, 21.8, 17500.0, 6800.0, 40°54′53″N 74°09′46″W
Mayor found  Andre Sayegh
website found None
Torrance, California, 145182.0, 145438.0, 0.18, 20.5, 53.1, 7180.0, 2770.0, 33°50′06″N 118°20′29″W
M

Mayor found  Ray O'Connell
website found None
Richardson, Texas, 120981.0, 99223.0, 21.93, 28.6, 74.1, 3963.0, 1530.0, 32°58′20″N 96°42′29″W
Mayor found  City Council
website found None
Odessa, Texas, 120568.0, 99940.0, 20.64, 45.2, 117.1, 2608.0, 1007.0, 31°53′02″N 102°20′28″W
Mayor found  City Council
website found None
Arvada, Colorado, 120492.0, 106433.0, 13.21, 38.6, 100.0, 3043.0, 1175.0, 39°50′01″N 105°09′01″W
Mayor found  Marc Williams
website found None
Cambridge, Massachusetts, 118977.0, 105162.0, 13.14, 6.4, 16.6, 17289.0, 6675.0, 42°22′34″N 71°07′07″W
website found None
Sugar Land, Texas, 118600.0, 91252.0, 29.97, 34.0, 88.1, 3488.0, 1347.0, 29°35′58″N 95°36′51″W
Mayor found  City Council
website found None
Beaumont, Texas, 118428.0, 118296.0, 0.11, 82.1, 212.6, 1441.0, 556.0, 30°05′06″N 94°08′43″W
Mayor found  City Council
website found None
Lansing, Michigan, 118427.0, 114297.0, 3.61, 39.1, 101.3, 2967.0, 1146.0, 42°42′51″N 84°33′33″W
Mayor found  Andy Schor
website found

website found None
San Mateo, California, 105025.0, 97207.0, 8.04, 12.1, 31.3, 8592.0, 3317.0, 37°33′37″N 122°18′38″W
Mayor found  Diane Papan
website found None
Green Bay, Wisconsin, 104879.0, 104057.0, 0.79, 45.4, 117.6, 2316.0, 894.0, 44°31′15″N 87°59′03″W
website found None
Wichita Falls, Texas, 104576.0, 104553.0, 0.02, 72.2, 187.0, 1450.0, 560.0, 33°54′24″N 98°31′33″W
Mayor found  Stephen Santellana
website found None
Sparks, Nevada, 104246.0, 90264.0, 15.49, 35.9, 93.0, 2810.0, 1080.0, 39°33′16″N 119°44′08″W
Mayor found  Ron Smith (
website found None
Lakewood, New Jersey, 104157.0, 92843.0, 12.19, 24.7, 64.0, 4079.0, 1575.0, 40°04′38″N 74°12′01″W
Mayor found  Raymond Coles (
website found None
Burbank, California, 103695.0, 103340.0, 0.34, 17.4, 45.1, 6003.0, 2318.0, 34°11′24″N 118°19′35″W
Mayor found  Emily Gabel-Luddy
website found burbankca.gov
Rialto, California, 103440.0, 99171.0, 4.3, 22.3, 57.8, 4633.0, 1789.0, 34°06′42″N 117°23′18″W
Mayor found  City council
website fou

Converting to csv file so that we can upload to googles bigQuerry

In [4]:
import pandas as pd
dataframe = pd.DataFrame.from_dict(data)
dataframe.to_csv('cities.csv')

In [7]:
from google.cloud import bigquery
client = bigquery.Client(project='alpha-8448e')
query = "SELECT  state, count(city) as City_Count FROM `alpha-8448e.USCities.Cities101` group by state;"
dataset = client.dataset('USCities')
table = dataset.table(name='Cities101')
job = client.run_async_query('my-job', query)
job.destination = table
job.write_disposition= 'WRITE_TRUNCATE'
job.begin()

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started